In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

from numba import njit
from pandas.plotting import scatter_matrix
from scipy.stats import norm, uniform, gaussian_kde, multivariate_normal
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

# 警告メッセージを非表示
import warnings
warnings.filterwarnings("ignore")

In [20]:
# データの取得
data = pd.read_excel(r"C:\Users\Kohsu\OneDrive\ドキュメント\東洋大学3年次\研究発表大会\【推計用データ】『サーチ理論に基づく日本の労働市場分析：マッチング関数による時期別検証』，研究発表大会.xlsx", header=[0])
data = data.rename(columns = {'Unnamed: 0': 'date'})
print(data)

# データの設定
new_employ = data['就職件数(万件)']
recruit_lag1 = data['新規求人数(万人)'].shift(1)
unemploy_lag1 = data['完全失業者(万人)'].shift(1)

d_antei = data['ダミー変数(安定成長期)']
d_bubble = data['ダミー変数(バブル期)']
d_houkai = data['ダミー変数(バブル崩壊以後)']
d_lehman = data['ダミー変数(リーマン・ショック期)']
d_korona = data['ダミー変数(コロナショック期)']
d_koizumi = data['ダミー変数(小泉)']
trend = data['トレンド変数']

    date  就職件数(万件)  新規求人数(万人)  完全失業者(万人)  トレンド変数  ダミー変数(安定成長期)  ダミー変数(バブル期)  \
0   1963   16.3749    35.9792         59       1             0            0   
1   1964   16.1938    39.3330         54       2             0            0   
2   1965   14.7594    30.8551         57       3             0            0   
3   1966   14.9272    36.0993         65       4             0            0   
4   1967   15.1075    43.6562         63       5             0            0   
..   ...       ...        ...        ...     ...           ...          ...   
57  2020   10.3156    75.0864        162      58             0            0   
58  2021   10.4935    78.1572        192      59             0            0   
59  2022   10.1431    86.6369        195      60             0            0   
60  2023   10.1861    86.6937        179      61             0            0   
61  2024    9.7271    83.6071        178      62             0            0   

    ダミー変数(バブル崩壊以後)  ダミー変数(リーマン・ショック期)  ダミー変数(コロナショッ

In [21]:
# 変数の対数化
log_new_employ = np.log(new_employ)
log_unemploy_lag1 = np.log(unemploy_lag1)
log_recruit_lag1 = np.log(recruit_lag1)
# print(log_new_employ)
# print(log_unemploy_lag1)
# print(log_recruit_lag1)

In [22]:
# データフレーム
df = pd.DataFrame({
    "date": data["date"],
    "log_new_employ": log_new_employ,
    "log_unemploy_lag1": log_unemploy_lag1,
    "log_recruit_lag1": log_recruit_lag1,
    "trend": trend,
    "d_antei": d_antei,
    "d_bubble": d_bubble,
    "d_houkai": d_houkai,
    "d_lehman": d_lehman,
    "d_korona": d_korona,
    "d_koizumi":d_koizumi
})

df = df.dropna()
df = df.set_index("date")
print(df)

      log_new_employ  log_unemploy_lag1  log_recruit_lag1  trend  d_antei  \
date                                                                        
1964        2.784628           4.077537          3.582941      2        0   
1965        2.691880           3.988984          3.672064      3        0   
1966        2.703185           4.043051          3.429302      4        0   
1967        2.715191           4.174387          3.586273      5        0   
1968        2.730705           4.143135          3.776345      6        0   
...              ...                ...               ...    ...      ...   
2020        2.333657           5.117994          4.563064     58        0   
2021        2.350756           5.087596          4.318639     59        0   
2022        2.316794           5.257495          4.358722     60        0   
2023        2.321024           5.273000          4.461726     61        0   
2024        2.274916           5.187386          4.462381     62        0   

In [23]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [24]:
# ADF検定

import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller

ADF_log_new_employ = adfuller(df['log_new_employ'])
print(f'log_new_employのADF統計量: {ADF_log_new_employ[0]:.4f}')
print(f'log_new_employのP値: {ADF_log_new_employ[1]:.4f}')

ADF_log_unemploy_lag1 = adfuller(df['log_unemploy_lag1'])
print(f'log_unemploy_lag1のADF統計量: {ADF_log_unemploy_lag1[0]:.4f}')
print(f'log_unemploy_lag1のP値: {ADF_log_unemploy_lag1[1]:.4f}')

ADF_log_recruit_lag1 = adfuller(df['log_recruit_lag1'])
print(f'log_unemploy_lag1のADF統計量: {ADF_log_unemploy_lag1[0]:.4f}')
print(f'log_recruit_lag1のP値: {ADF_log_recruit_lag1[1]:.4f}')

log_new_employのADF統計量: -1.2835
log_new_employのP値: 0.6367
log_unemploy_lag1のADF統計量: -2.0283
log_unemploy_lag1のP値: 0.2743
log_unemploy_lag1のADF統計量: -2.0283
log_recruit_lag1のP値: 0.7916


In [25]:
# 階差を取った場合のADF検定

import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller

df_diff = pd.DataFrame({
    "log_new_employ_diff": df['log_new_employ'].diff(),
    "log_unemploy_lag1_diff": df['log_unemploy_lag1'].diff(),
    "log_recruit_lag1_diff": df['log_recruit_lag1'].diff()
})

ADF_log_new_employ_diff = adfuller(df_diff['log_new_employ_diff'].dropna())
print(f'log_new_employ_diffのADF統計量: {ADF_log_new_employ_diff[0]:.4f}')
print(f'log_new_employ_diffのP値: {ADF_log_new_employ_diff[1]:.4f}')

ADF_log_unemploy_lag1_diff = adfuller(df_diff['log_unemploy_lag1_diff'].dropna())
print(f'log_unemploy_lag1_diffのADF統計量: {ADF_log_unemploy_lag1_diff[0]:.4f}')
print(f'log_unemploy_lag1_diffのP値: {ADF_log_unemploy_lag1_diff[1]:.4f}')

ADF_log_recruit_lag1_diff = adfuller(df_diff['log_recruit_lag1_diff'].dropna())
print(f'log_recruit_lag1_diffのADF統計量: {ADF_log_recruit_lag1_diff[0]:.4f}')
print(f'log_recruit_lag1_diffのP値: {ADF_log_recruit_lag1_diff[1]:.4f}')

log_new_employ_diffのADF統計量: -4.9766
log_new_employ_diffのP値: 0.0000
log_unemploy_lag1_diffのADF統計量: -5.0823
log_unemploy_lag1_diffのP値: 0.0000
log_recruit_lag1_diffのADF統計量: -5.3412
log_recruit_lag1_diffのP値: 0.0000


In [26]:
# # 全期間(1964-2024年)

# df_all = df.loc[1964:2024].copy()

# df_all['y_all_diff'] = (df_all['log_new_employ'] - df_all['log_unemploy_lag1']).diff()
# df_all['x_all_diff'] = (df_all['log_recruit_lag1'] - df_all['log_unemploy_lag1']).diff()

# regre = 'y_all_diff ~ x_all_diff + d_antei + d_bubble + d_houkai + d_lehman + x_all_diff:d_antei + x_all_diff:d_bubble + x_all_diff:d_houkai + x_all_diff:d_lehman'
# res = smf.ols(regre, data=df_all).fit()

# print(res.summary())

In [27]:
# # 規模に関して収穫一定の仮定を置いた場合

# df_all = df.loc[1964:2024].copy()

# df_all['y_all_diff'] = (df_all['log_new_employ'] - df_all['log_unemploy_lag1']).diff()
# df_all['x_all_diff'] = (df_all['log_recruit_lag1'] - df_all['log_unemploy_lag1']).diff()

# # モデル1
# model1 = 'y_all_diff ~ x_all_diff'
# res_model1 = smf.ols(model1, data=df_all).fit()
# print(res_model1.summary())

# # モデル2(安定成長期+バブル期)
# model2 = 'y_all_diff ~ x_all_diff + d_antei + d_bubble + x_all_diff:d_antei + x_all_diff:d_bubble'
# res_model2 = smf.ols(model2, data=df_all).fit()
# print(res_model2.summary())

# # モデル3(バブル崩壊以後+リーマン・ショック期+コロナショック期)
# model3 = 'y_all_diff ~ x_all_diff + d_houkai + d_lehman + d_korona + x_all_diff:d_houkai + x_all_diff:d_lehman + x_all_diff:d_korona'
# res_model3 = smf.ols(model3, data=df_all).fit()
# print(res_model3.summary())

In [28]:
# # 規模に関する収穫の仮定を置かない場合

# df_all = df.loc[1964:2024].copy()

# y = df_all['log_new_employ'].diff()
# x1 = df_all['log_recruit_lag1'].diff()
# x2 = df_all['log_unemploy_lag1'].diff()

# # モデル1
# model1 = 'y ~ x1 + x2'
# res_model1 = smf.ols(model1, data=df_all).fit()
# print(res_model1.summary())

# # モデル2(安定成長期+バブル期)
# model2 = 'y ~ x1 + x2 + d_antei + d_bubble + x1:d_antei + x1:d_bubble + x2:d_antei + x2:d_bubble'
# res_model2 = smf.ols(model2, data=df_all).fit()
# print(res_model2.summary())

# # モデル3(バブル崩壊以後+リーマン・ショック期+コロナショック期)
# model3 = 'y ~ x1 + x2 + d_houkai + d_lehman + d_korona + x1:d_houkai + x1:d_lehman + x1:d_korona + x2:d_houkai + x2:d_lehman + x2:d_korona'
# res_model3 = smf.ols(model3, data=df_all).fit()
# print(res_model3.summary())

In [29]:
# 研究発表大会用(規模に関して収穫一定)

df_all = df.loc[1964:2024].copy()

df_all['y_all_diff'] = (df_all['log_new_employ'] - df_all['log_unemploy_lag1']).diff()
df_all['x_all_diff'] = (df_all['log_recruit_lag1'] - df_all['log_unemploy_lag1']).diff()

# モデル1
model1 = 'y_all_diff ~ x_all_diff'
res_model1 = smf.ols(model1, data=df_all).fit()
print(res_model1.summary())

# モデル2(安定成長期+バブル期+バブル崩壊以後)
model2 = 'y_all_diff ~ x_all_diff + d_antei + d_bubble + d_houkai + x_all_diff:d_antei + x_all_diff:d_bubble + x_all_diff:d_houkai'
res_model2 = smf.ols(model2, data=df_all).fit()
print(res_model2.summary())

# モデル3(小泉構造改革期+リーマン・ショック期+コロナショック期)
model3 = 'y_all_diff ~ x_all_diff + d_koizumi + d_lehman + d_korona + x_all_diff:d_koizumi + x_all_diff:d_lehman + x_all_diff:d_korona'
res_model3 = smf.ols(model3, data=df_all).fit()
print(res_model3.summary())

                            OLS Regression Results                            
Dep. Variable:             y_all_diff   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     16.96
Date:                Fri, 02 Jan 2026   Prob (F-statistic):           0.000123
Time:                        22:55:46   Log-Likelihood:                 69.210
No. Observations:                  60   AIC:                            -134.4
Df Residuals:                      58   BIC:                            -130.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0260      0.010     -2.595      0.0

In [30]:
!pip install linearmodels

In [31]:
# ホワイトの検定

from sklearn. linear_model import LinearRegression
from statsmodels. stats . diagnostic import het_white
import statsmodels. api as sm

labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']

white_test1 = het_white(res_model1.resid, res_model1.model.exog )
print (dict(zip(labels, white_test1)))

white_test2 = het_white(res_model2.resid, res_model2.model.exog )
print (dict(zip(labels, white_test2)))

white_test3 = het_white(res_model3.resid, res_model3.model.exog )
print (dict(zip(labels, white_test3)))

{'Test Statistic': np.float64(7.776240621128898), 'Test Statistic p-value': np.float64(0.020483813080612135), 'F-Statistic': np.float64(4.243717042550532), 'F-Test p-value': np.float64(0.01913952336884591)}
{'Test Statistic': np.float64(13.05541506941173), 'Test Statistic p-value': np.float64(0.28971894835328404), 'F-Statistic': np.float64(1.2135389848155016), 'F-Test p-value': np.float64(0.3041203808055882)}
{'Test Statistic': np.float64(12.861469149082199), 'Test Statistic p-value': np.float64(0.23152072139495763), 'F-Statistic': np.float64(1.3369359989138423), 'F-Test p-value': np.float64(0.23805612159239545)}


In [32]:
# 不均一分散修正標準誤差

res_model1_robust = res_model1.get_robustcov_results(cov_type='HC3')
print(res_model1_robust.summary())

res_model2_robust = res_model2.get_robustcov_results(cov_type='HC3')
print(res_model2_robust.summary())

res_model3_robust = res_model3.get_robustcov_results(cov_type='HC3')
print(res_model3_robust.summary())

                            OLS Regression Results                            
Dep. Variable:             y_all_diff   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     12.10
Date:                Fri, 02 Jan 2026   Prob (F-statistic):           0.000962
Time:                        22:55:52   Log-Likelihood:                 69.210
No. Observations:                  60   AIC:                            -134.4
Df Residuals:                      58   BIC:                            -130.2
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0260      0.010     -2.573      0.0

In [33]:
# 操作変数法(モデル1)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from linearmodels.iv import IV2SLS
from scipy.stats import chi2, gaussian_kde, multivariate_normal, norm, uniform

# 警告メッセージを非表示
import warnings
warnings.filterwarnings("ignore")

df_all = df.loc[1964:2024].copy()

df_all['y_all_diff'] = (df_all['log_new_employ'] - df_all['log_unemploy_lag1']).diff()
df_all['x_all_diff'] = (df_all['log_recruit_lag1'] - df_all['log_unemploy_lag1']).diff()

df_all['z1'] = df_all['x_all_diff'].shift(1)
df_all['z2'] = df_all['x_all_diff'].shift(2)

df_all = df_all.dropna()

iv = 'y_all_diff ~ 1 + [x_all_diff ~ z1 + z2]'

iv_result = IV2SLS.from_formula(iv, data=df_all).fit()
print(iv_result.summary)
print(iv_result.first_stage)

# # P値
# from scipy import stats

# t =       # t値
# df = 59      # 自由度

# p_value = 2 * (1 - stats.t.cdf(abs(t), df))
# print(p_value)


                          IV-2SLS Estimation Summary                          
Dep. Variable:             y_all_diff   R-squared:                      0.0454
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0284
No. Observations:                  58   F-statistic:                    8.6661
Date:                Fri, Jan 02 2026   P-value (F-stat)                0.0032
Time:                        22:55:52   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -0.0261     0.0112    -2.3357     0.0195     -0.0481     -0.0042
x_all_diff     0.5048     0.1715     2.9438     0.00

In [34]:
# DWH検定

import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

iv_result.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 4.6602
P-value: 0.0352
Distributed: F(1,55)
WaldTestStatistic, id: 0x1f82b73aed0

In [35]:
# 規模に関して収穫一定の仮定を置いた場合(1990-2024年)

df_all = df.loc[1990:2024].copy()

df_all['y_all_diff'] = (df_all['log_new_employ'] - df_all['log_unemploy_lag1']).diff()
df_all['x_all_diff'] = (df_all['log_recruit_lag1'] - df_all['log_unemploy_lag1']).diff()

# モデル1
model1 = 'y_all_diff ~ x_all_diff'
res_model1 = smf.ols(model1, data=df_all).fit()
print(res_model1.summary())

# モデル3(小泉構造改革期+リーマン・ショック期+コロナショック期)
model3 = 'y_all_diff ~ x_all_diff + d_koizumi + d_lehman + d_korona + x_all_diff:d_koizumi + x_all_diff:d_lehman + x_all_diff:d_korona'
res_model3 = smf.ols(model3, data=df_all).fit()
print(res_model3.summary())

                            OLS Regression Results                            
Dep. Variable:             y_all_diff   R-squared:                       0.110
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     3.943
Date:                Fri, 02 Jan 2026   Prob (F-statistic):             0.0557
Time:                        22:55:52   Log-Likelihood:                 39.213
No. Observations:                  34   AIC:                            -74.43
Df Residuals:                      32   BIC:                            -71.37
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0098      0.014     -0.728      0.4

In [36]:
# トレンド変数によるマッチング効率性

df_all = df.loc[1964:2024].copy()

df_all['y_all_diff'] = (df_all['log_new_employ'] - df_all['log_unemploy_lag1']).diff()
df_all['x_all_diff'] = (df_all['log_recruit_lag1'] - df_all['log_unemploy_lag1']).diff()

# モデル
model_trend = 'y_all_diff ~ x_all_diff + trend'
res_model_trend = smf.ols(model_trend, data=df_all).fit()
print(res_model_trend.summary())

                            OLS Regression Results                            
Dep. Variable:             y_all_diff   R-squared:                       0.246
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                     9.277
Date:                Fri, 02 Jan 2026   Prob (F-statistic):           0.000325
Time:                        22:55:52   Log-Likelihood:                 69.970
No. Observations:                  60   AIC:                            -133.9
Df Residuals:                      57   BIC:                            -127.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0490      0.021     -2.283      0.0